In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.src.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')


In [5]:
data = pd.read_csv("datasets/crop_data.csv")

X = data[["N", "P", "K", "temperature", "humidity", "ph"]]
y = data["label"]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

y_categorical = to_categorical(y_encoded)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.2, random_state=42
)


model = Sequential()

model.add(Dense(256, input_dim=6, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(32, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(y_categorical.shape[1], activation="softmax"))


In [13]:
from keras.src.callbacks import EarlyStopping

optimizer = Adam(learning_rate=0.001)
model.compile(
    loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)

model.summary()
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
)


In [14]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовых данных: {test_accuracy * 100:.2f}%")

plt.plot(history.history["accuracy"], label="Точность на обучающем наборе")
plt.plot(history.history["val_accuracy"], label="Точность на валидационном наборе")
plt.xlabel("Эпоха")
plt.ylabel("Точность")
plt.legend()
plt.show()

plt.plot(history.history["loss"], label="Потери на обучающем наборе")
plt.plot(history.history["val_loss"], label="Потери на валидационном наборе")
plt.xlabel("Эпоха")
plt.ylabel("Потери")
plt.legend()
plt.show()


In [16]:
model.save("crop.h5")


In [19]:
new_sample = np.array([[100, 40, 50, 25, 80, 6.0]]) 
print(new_sample)

new_sample_scaled = scaler.transform(new_sample)
print(new_sample_scaled)
prediction = model.predict(new_sample_scaled)
predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])

print(f"Рекомендуемая культура для посева: {predicted_class[0]}")
